# 1. Introduction 

The goal of this project is to develop a content-based recommendation for movies and TV shows on Netflix Using *cast, director, country, rating and genres* as features.


# 2. Imports

In [5]:
import numpy as np
import pandas as pd
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darshanbafna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 3. Loading data

In [6]:
data = pd.read_csv('netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [8]:
data = data.dropna(subset=['cast', 'country', 'rating'])

# 4. Developing Recommendation Engine using *cast, director, country, rating and genres*

In [9]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [10]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [11]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [12]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,29105,29106,29107,29108,29109,29110,29111,29112,29113,29114
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1
        
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [14]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 4.1. Recommending Movies

In [16]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
507,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada",PG-13,"Horror Movies, Thrillers",0.424437
3223,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.416667
603,The Conjuring 2,James Wan,"Patrick Wilson, Vera Farmiga, Madison Wolfe, F...","Canada, United States, United Kingdom",R,Horror Movies,0.412479
1832,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.400892
3728,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.395285


In [17]:
recommender("Child's Play")

,title,director,cast,country,rating,listed_in,cos_sim
3689,Cult of Chucky,Don Mancini,"Fiona Dourif, Michael Therriault, Adam Hurtig,...",United States,R,Horror Movies,0.352941
1461,Wildling,Fritz Böhm,"Bel Powley, Brad Dourif, Liv Tyler, Collin Kel...",United States,R,"Horror Movies, Independent Movies, Sci-Fi & Fa...",0.350070
4768,Stephanie,Akiva Goldsman,"Shree Cooks, Frank Grillo, Anna Torv",United States,R,Horror Movies,0.342997
4434,Nova: Ultimate Mars Challenge,Gail Willumsen,Lance Lewman,United States,TV-G,Movies,0.325396
3160,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,84 min,Movies,0.325396


In [18]:
recommender('Charlie and the Chocolate Factory')

,title,director,cast,country,rating,listed_in,cos_sim
3835,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.394055
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.390095
4802,Sweeney Todd: The Demon Barber of Fleet Street,Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickma...","United States, United Kingdom",R,"Dramas, Horror Movies, Music & Musicals",0.382692
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.361158
1547,A Shaun the Sheep Movie: Farmageddon,"Richard Phelan, Will Becher","Justin Fletcher, John Sparkes, Amalia Vitale, ...","United Kingdom, France, Belgium, Ireland, Unit...",G,"Children & Family Movies, Comedies",0.354005


In [19]:
recommender('Wild Child')

,title,director,cast,country,rating,listed_in,cos_sim
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.440959
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.408248
1130,The Kissing Booth 2,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.394405
122,The Kissing Booth 3,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.381881
2501,The Guernsey Literary and Potato Peel Pie Society,Mike Newell,"Lily James, Michiel Huisman, Penelope Wilton, ...","United Kingdom, France, United States",TV-PG,"Dramas, Romantic Movies",0.381881


In [20]:
recommender("Dr. Seuss' The Cat in the Hat")

,title,director,cast,country,rating,listed_in,cos_sim
4419,NOVA: Bird Brain,NaN,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.436436
1114,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.436436
4758,Spy Kids 3: Game Over,Robert Rodriguez,"Daryl Sabara, Sylvester Stallone, Ricardo Mont...",United States,PG,"Children & Family Movies, Comedies",0.422577
2245,Prince of Peoria: A Christmas Moose Miracle,Jon Rosenbaum,"Gavin Lewis, Theodore Barnes, Shelby Simmons, ...",United States,TV-G,"Children & Family Movies, Comedies",0.422577
1363,What a Girl Wants,Dennie Gordon,"Amanda Bynes, Colin Firth, Kelly Preston, Eile...","United States, United Kingdom",PG,"Children & Family Movies, Comedies",0.414039


In [21]:
recommender('Hook')

,title,director,cast,country,rating,listed_in,cos_sim
1918,Sotus The Series,NaN,"Sangpotirat Perawat, Ruangroj Prachaya",Thailand,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
843,Little Things,NaN,"Dhruv Sehgal, Mithila Palkar",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
1898,Roonpi Secret Love,NaN,"Kanyawee Songmuang, Korapat Kirdpan",Thailand,TV-14,"International TV Shows, Romantic TV Shows, Tee...",0.405554
200,Let's Fight Ghost,Kongkiat Khomsiri,"Suppapong Udomkaewkanjana, Patchanan Jiajirach...",Thailand,TV-14,"International TV Shows, Romantic TV Shows, TV ...",0.381771
1477,Highland: Thailand's Marijuana Awakening,NaN,Sebastian Perry,Thailand,TV-MA,"Docuseries, International TV Shows",0.374634


In [22]:
recommender('Holidate')

,title,director,cast,country,rating,listed_in,cos_sim
2722,When We First Met,Ari Sandel,"Adam DeVine, Alexandra Daddario, Shelley Henni...",United States,TV-14,"Comedies, Romantic Movies",0.467099
3844,First Impression,Arthur Muhammad,"Lamman Rucker, Lisa Arrindell Anderson, Elise ...",United States,TV-14,"Comedies, Romantic Movies",0.408248
2855,Chadi Jawani Budhe Nu,NaN,Anand,India,TV-PG,"Comedies, International Movies, Romantic Movies",0.390360
1783,Holly Star,Michael A. Nickles,"Katlyn Carlson, Brian Muller, Teya Patt, Pamel...",United States,PG,"Children & Family Movies, Comedies, Romantic M...",0.389249
4938,The Good Catholic,Paul Shoulberg,"Zachary Spicer, Wrenn Schmidt, Danny Glover, J...",United States,PG-13,"Comedies, Dramas, Romantic Movies",0.389249


## 4.2. Recommending TV shows

In [23]:
recommender('After Life')

,title,director,cast,country,rating,listed_in,cos_sim
1699,Extras,NaN,"Ricky Gervais, Stephen Merchant, Ashley Jensen...","United Kingdom, United States",TV-MA,"British TV Shows, TV Comedies",0.564076
1816,Mary Portas: Secret Shopper,NaN,Mary Portas,United Kingdom,TV-14,"British TV Shows, International TV Shows, Real...",0.505076
1636,"Big Dreams, Small Spaces",NaN,Monty Don,United Kingdom,TV-G,"British TV Shows, International TV Shows, Real...",0.505076
1717,Genius of the Ancient World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076
1718,Genius of the Modern World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076


In [24]:
recommender('Anne with an E')

,title,director,cast,country,rating,listed_in,cos_sim
400,Can You Hear Me?,NaN,"Mélissa Bédard, Ève Landry, Florence Longpré",Canada,TV-MA,"International TV Shows, TV Comedies, TV Dramas",0.507093
621,Restaurants on the Edge,NaN,"Nick Liberato, Karin Bohn, Dennis Prescott",Canada,TV-14,"International TV Shows, Reality TV",0.445435
1739,Heavy Rescue: 401,NaN,Dave Pettitt,Canada,TV-MA,"International TV Shows, Reality TV",0.436436
750,Hip-Hop Evolution,NaN,Shad Kabango,Canada,TV-MA,"Docuseries, International TV Shows",0.436436
1637,Bitten,NaN,"Laura Vandervoort, Greyston Holt, Greg Bryk, S...",Canada,TV-MA,"International TV Shows, TV Dramas, TV Horror",0.428571


In [25]:
recommender('Gilmore Girls')

,title,director,cast,country,rating,listed_in,cos_sim
1532,Gilmore Girls: A Year in the Life,NaN,"Lauren Graham, Alexis Bledel, Kelly Bishop, Sc...",United States,TV-14,"TV Comedies, TV Dramas, Teen TV Shows",0.789474
1411,No Tomorrow,NaN,"Joshua Sasse, Tori Anderson, Jonathan Langdon,...",United States,TV-PG,"Romantic TV Shows, TV Comedies, TV Dramas",0.445399
395,DASH & LILY,NaN,"Midori Francis, Austin Abrams, Dante Brown, Tr...",United States,TV-14,"Romantic TV Shows, TV Comedies, TV Dramas",0.435286
1233,Hot Date,NaN,"Emily Axford, Brian Murphy",United States,TV-MA,"Romantic TV Shows, TV Comedies",0.433555
441,Emily in Paris,NaN,"Lily Collins, Philippine Leroy-Beaulieu, Ashle...",United States,TV-MA,"Romantic TV Shows, TV Comedies, TV Dramas",0.414644


In [26]:
recommender('Friends')

,title,director,cast,country,rating,listed_in,cos_sim
1352,Episodes,NaN,"Matt LeBlanc, Tamsin Greig, Stephen Mangan, Ka...","United Kingdom, United States",TV-MA,"Classic & Cult TV, TV Comedies",0.522233
1858,Pee-wee's Playhouse,NaN,Paul Reubens,United States,TV-PG,"Classic & Cult TV, Kids' TV, TV Comedies",0.492366
1172,The Fix,NaN,"Jimmy Carr, D.L. Hughley, Katherine Ryan",United States,TV-MA,TV Comedies,0.455842
1310,The Honeymoon Stand Up Special,NaN,"Natasha Leggero, Moshe Kasher",United States,TV-MA,"Stand-Up Comedy & Talk Shows, TV Comedies",0.455842
1605,Adam Ruins Everything,NaN,"Adam Conover, Adam Lustick, Emily Axford",United States,TV-14,TV Comedies,0.455842


In [27]:
recommender('Love on the Spectrum')

,title,director,cast,country,rating,listed_in,cos_sim
1659,Cheapest Weddings,NaN,Melanie Vallejo,Australia,TV-14,"International TV Shows, Reality TV",0.771517
1669,Court Justice,NaN,Chris Bourke,Australia,TV-MA,"Crime TV Shows, Docuseries, International TV S...",0.714286
412,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,TV-PG,"International TV Shows, Reality TV",0.714286
1573,72 Dangerous Places to Live,NaN,Mitch Ryan,Australia,TV-PG,"Docuseries, International TV Shows, Science & ...",0.714286
1264,72 Dangerous Animals: Asia,NaN,Bob Brisbane,Australia,TV-14,"Docuseries, International TV Shows, Science & ...",0.714286


In [28]:
recommender('13 Reasons Why')

,title,director,cast,country,rating,listed_in,cos_sim
934,13 Reasons Why: Beyond the Reasons,NaN,"Dylan Minnette, Katherine Langford, Kate Walsh...",United States,TV-MA,"Crime TV Shows, Docuseries",0.453557
1413,Frequency,NaN,"Peyton List, Riley Smith, Mekhi Phifer, Devin ...",United States,TV-14,"Crime TV Shows, TV Dramas, TV Mysteries",0.418330
941,MINDHUNTER,NaN,"Jonathan Groff, Holt McCallany, Anna Torv, Cot...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.404520
281,The Sinner,NaN,"Jessica Biel, Bill Pullman, Christopher Abbott...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.404520
1290,Life Sentence,NaN,Lucy Hale,United States,TV-14,TV Dramas,0.400000


In [29]:
recommender('Derry Girls')

,title,director,cast,country,rating,listed_in,cos_sim
1513,Crazyhead,NaN,"Cara Theobold, Susan Wokoma, Riann Steele, Ari...",United Kingdom,TV-MA,"British TV Shows, International TV Shows, TV C...",0.457604
1322,James Acaster: Repertoire,Dan Lucchesi,James Acaster,United Kingdom,TV-14,"British TV Shows, International TV Shows, Stan...",0.445435
1862,Planet Earth: The Complete Collection,Alastair Fothergill,David Attenborough,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.445435
639,Nadiya’s Time to Eat,NaN,Nadiya Hussain,United Kingdom,TV-G,"British TV Shows, International TV Shows, Real...",0.445435
1828,My Hotter Half,NaN,Melvin Odoom,United Kingdom,TV-PG,"British TV Shows, International TV Shows, Real...",0.445435


In [30]:
recommender('Breaking Bad')

,title,director,cast,country,rating,listed_in,cos_sim
721,Better Call Saul,NaN,"Bob Odenkirk, Jonathan Banks, Michael McKean, ...",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.554700
143,The Assassination of Gianni Versace,NaN,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.474342
1721,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.441942
1113,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.441942
1882,Prohibition: A Film by Ken Burns and Lynn Novick,NaN,Peter Coyote,United States,TV-PG,"Crime TV Shows, Docuseries",0.408248


In [31]:
recommender('Stranger Things')

,title,director,cast,country,rating,listed_in,cos_sim
1388,Beyond Stranger Things,NaN,"Jim Rash, Matt Duffer, Ross Duffer, Shawn Levy...",United States,TV-14,"Stand-Up Comedy & Talk Shows, TV Mysteries, TV...",0.752618
230,Prank Encounters,NaN,Gaten Matarazzo,United States,TV-MA,"Reality TV, TV Comedies, TV Horror",0.348155
1740,Helix,NaN,"Billy Campbell, Hiroyuki Sanada, Kyra Zagorsky...","United States, Canada",TV-MA,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.301511
997,Scare Tactics,NaN,"Tracy Morgan, Lauren Ash",United States,TV-MA,"Reality TV, TV Comedies, TV Horror",0.301511
45,Brand New Cherry Flavor,NaN,"Rosa Salazar, Eric Lange, Catherine Keener, Je...",United States,TV-MA,"TV Dramas, TV Horror, TV Mysteries",0.295656
